In [5]:
%load_ext autoreload
%autoreload 2

In [34]:
from feast import FeatureStore, Entity, FeatureView, Feature, ValueType, FileSource, RepoConfig
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage
from datetime import datetime, timedelta
import pandas as pd
import os

In [39]:
os.system("cd ..")

0

In [35]:
os.system("cd /usr/src/datanoob/repository && feast apply")

512

In [32]:
repo_path = "../repository/feature_store"

# fs = FeatureStore(config=RepoConfig(registry=f"{repo_path}/registry.db", 
#                                     project="mlops_regression", 
#                                     provider="local",
#                                     # online_store={"path":"../repository/feature_store/online_store.db"},
#                                     # online_store=dict(path="../repository/feature_store/online_store.db"),
#                                     # online_store="../repository/feature_store/online_store.db"
#                                    )
#                  )

repo_path = "../repository"
fs = FeatureStore(repo_path=f"{repo_path}")

house = Entity(name="house_id", value_type=ValueType.INT64, description="house_id",)

toxic_monthly_stats = FileSource(
    # path=f"{repo_path}/toxic.parquet",
    path=f"usr/src/datanoob/repository/feature_store/toxic.parquet",
    event_timestamp_column="event_timestamp",
    # created_timestamp_column="created",
)

toxic_monthly_stats_view = FeatureView(
    name="toxic_monthly_stats",
    entities=["house_id"],
    ttl=timedelta(seconds=86400 * 1),
    features=[
        Feature(name="NOX", dtype=ValueType.FLOAT),
        # Feature(name="acc_rate", dtype=ValueType.FLOAT),
        # Feature(name="avg_daily_trips", dtype=ValueType.INT64),
    ],
    batch_source=toxic_monthly_stats,
)
fs.apply([toxic_monthly_stats_view, house]) # register entity and feature view
# fs.materialize(
#     start_date=datetime.utcnow() - timedelta(hours=3), end_date=datetime.utcnow() - timedelta(minutes=10)
# )
# fs.materialize_incremental(end_date=datetime.now())

In [33]:
fs.materialize_incremental(end_date=datetime.now())

Materializing 1 feature views to 2022-07-16 20:01:45+00:00 into the sqlite online store.

toxic_monthly_stats from 2022-07-15 20:01:45+00:00 to 2022-07-16 20:01:45+00:00:


FileNotFoundError: /usr/src/datanoob/notebook/usr/src/datanoob/repository/feature_store/toxic.parquet

In [17]:
online_response = fs.get_online_features(
    features=[
        "toxic_monthly_stats:NOX",
        # "driver_hourly_stats:acc_rate",
        # "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[{"house_id": 2}],
)
online_response_dict = online_response.to_dict()
online_response_dict

{'house_id': [2], 'NOX': [None]}

In [26]:
# Getting our FeatureStore
repo_path = "../repository"
store = FeatureStore(repo_path=f"{repo_path}")

# Reading our targets as an entity DataFrame
entity_df = pd.read_parquet(path=f"{repo_path}/feature_store/target.parquet")
entity_df

,house_id,event_timestamp,MEDV,load_dt
0,1,2022-04-01,24.0,2022-04-01
1,2,2022-04-01,21.6,2022-04-01
2,3,2022-04-01,34.7,2022-04-01
3,4,2022-04-01,33.4,2022-04-01
4,5,2022-04-01,36.2,2022-04-01
...,...,...,...,...
501,502,2022-04-01,22.4,2022-04-01
502,503,2022-04-01,20.6,2022-04-01
503,504,2022-04-01,23.9,2022-04-01
504,505,2022-04-01,22.0,2022-04-01


In [27]:
training_data = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "toxic_monthly_stats:NOX",
    ]
)

# Storing the dataset as a local file: your path
dataset = store.create_saved_dataset(
    from_=training_data,
    name="mlops_regression_dataset",
    storage=SavedDatasetFileStorage("../repository/metadata/training_dataset.parquet")
)

/opt/conda/lib/python3.9/site-packages/feast/feature_store.py:1115: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [ ]:
# fs get repo_pat = ymal
# FileSource path = full path

In [60]:
from google.protobuf.duration_pb2 import Duration
from feast import Entity, Feature, FeatureView, FileSource, ValueType

repo_path = "../repository"
fs = FeatureStore(repo_path=f"{repo_path}")

# Declaring an entity for the dataset
house = Entity(name="house_id", value_type=ValueType.INT64, description="house_id",)

toxic_monthly_stats = FileSource(
    # path=f"{repo_path}/toxic.parquet",
    path="/usr/src/datanoob/repository/feature_store/toxic.parquet",
    event_timestamp_column="event_timestamp",
)

toxic_monthly_stats_view = FeatureView(
    name="toxic_monthly_stats",
    entities=["house_id"],
    # ttl=timedelta(seconds=86400 * 1),
    ttl=Duration(seconds=86400 * 60),
    features=[
        Feature(name="NOX", dtype=ValueType.FLOAT),
    ],
    batch_source=toxic_monthly_stats,
)

# Declaring the source of the targets
target_source = FileSource(
    path="/usr/src/datanoob/repository/feature_store/target.parquet", 
    created_timestamp_column="event_timestamp")

# Defining the targets
target_fv = FeatureView(
    name="target_feature_view",
    entities=["house_id"],
    ttl=Duration(seconds=86400 * 60),
    features=[
        Feature(name="MEDV", dtype=ValueType.INT32)        
        ],    
    batch_source=target_source
)
fs.apply([toxic_monthly_stats_view, house, target_fv])

In [61]:
training_data = fs.get_historical_features(
    entity_df=entity_df,
    features=[
        "toxic_monthly_stats:NOX",
    ]
)

# Storing the dataset as a local file: your path
dataset = fs.create_saved_dataset(
    from_=training_data,
    name="mlops_regression_dataset",
    storage=SavedDatasetFileStorage("../repository/metadata/training_dataset.parquet"),
    # storage=SavedDatasetFileStorage("../repository/feature_store/training_dataset.parquet"),
)

/opt/conda/lib/python3.9/site-packages/feast/feature_store.py:1115: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [62]:
pd.read_parquet("../repository/metadata/training_dataset.parquet")

,house_id,event_timestamp,MEDV,load_dt,NOX
0,1,2022-04-01 00:00:00+00:00,24.0,2022-04-01,0.538
344,345,2022-04-01 00:00:00+00:00,31.2,2022-04-01,0.484
343,344,2022-04-01 00:00:00+00:00,23.9,2022-04-01,0.484
342,343,2022-04-01 00:00:00+00:00,16.5,2022-04-01,0.518
341,342,2022-04-01 00:00:00+00:00,32.7,2022-04-01,0.442
...,...,...,...,...,...
162,163,2022-04-01 00:00:00+00:00,50.0,2022-04-01,0.605
161,162,2022-04-01 00:00:00+00:00,50.0,2022-04-01,0.605
160,161,2022-04-01 00:00:00+00:00,27.0,2022-04-01,0.605
173,174,2022-04-01 00:00:00+00:00,23.6,2022-04-01,0.510


In [63]:
fs.materialize_incremental(end_date=datetime.now())

Materializing 2 feature views to 2022-07-16 20:48:22+00:00 into the sqlite online store.

toxic_monthly_stats from 2022-05-17 20:48:22+00:00 to 2022-07-16 20:48:22+00:00:


0it [00:00, ?it/s]


target_feature_view from 2022-05-17 20:48:22+00:00 to 2022-07-16 20:48:22+00:00:


0it [00:00, ?it/s]


In [66]:
fs.materialize(
    end_date=datetime.now(),
    start_date=datetime.now() - timedelta(days=700))

Materializing 2 feature views from 2020-08-15 20:51:43+00:00 to 2022-07-16 20:51:43+00:00 into the sqlite online store.

toxic_monthly_stats:


100%|██████████████████████████████████████████████████████████| 506/506 [00:00<00:00, 13913.01it/s]


target_feature_view:


100%|██████████████████████████████████████████████████████████| 506/506 [00:00<00:00, 15262.43it/s]


In [68]:
online_response = fs.get_online_features(
    features=[
        "toxic_monthly_stats:NOX",
        # "driver_hourly_stats:acc_rate",
        # "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[{"house_id": 2}, {"house_id": 345}, {"house_id": 344}],
)
online_response_dict = online_response.to_dict()
online_response_dict

{'house_id': [2, 345, 344],
 'NOX': [0.4690000116825104, 0.48399999737739563, 0.48399999737739563]}

In [56]:
# Defining our features names
feast_features = [
        "toxic_monthly_stats:NOX",
    ]

# Getting the latest features
features = fs.get_online_features(
    features=feast_features,    
    entity_rows=[{"house_id": 345}, {"house_id": 344}]
).to_dict()

# Converting the features to a DataFrame
features_df = pd.DataFrame.from_dict(data=features)
features_df

,house_id,NOX
0,345,None
1,344,None
